# Model calibration and prediction + plots for journal

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob, os
import math
import pickle
from datetime import datetime
import time
from collections import Counter
import forestci as fci
from sklearn.metrics import mean_squared_error, r2_score
%matplotlib inline
import scipy.stats as st
import matplotlib.patches as mpatches


'''Relevant paths'''
path_models = [] # trained models path
path = [] # data path
path_to_save_figures = [] # path to save results to 

# import functions
from results_plotting_functions import rmspe, alpha_accuracy_beta_rlh, count_entries_per_interval, predict_prob, std_calibrated, calibrated_prob, calibration_isotonic_regression_model





# Select datsets as per Nature NMI - see table in papper for refrence:
1. Group I: full_dataset 
2. Group II: standford_dataset
3.  Group III: oxford_dataset

In [ ]:
# select battery 
battery =  # group_1, group_2, group_3
adversarial = True

# Accuracy metrics

# Plotting functions

In [ ]:
def plot_results(y_hat, y_test, sem, sem_calibrated, z_value):
    # plot predicted results before and after re-calibration
    '''plotting the confidence interval and the prediction results'''
    x_axis = list(X_test.index.values)
    fig1 = plt.figure(figsize=(10, 10))

    # confidence interval
    
    # plt.errorbar(X_test.Cycle, y_hat, yerr=2 * sem, fmt='o', capsize=3, linewidth=2, ms=2)
    y_up = y_hat + z_value * sem
    y_low = y_hat - z_value * sem
    
    y_up_calibrated = y_hat + z_value * sem_calibrated
    y_low_calibrated = y_hat - z_value * sem_calibrated
    
    # accuracy zone calculationa
    y_acc_zone_up = y_test+alpha_acc_zone*y_test
    y_acc_zone_low = y_test-alpha_acc_zone*y_test


    #calculate predictions in accuracy zone
    acc_zone_entries, beta, rlh = alpha_accuracy_beta_rlh(y_test, y_hat, sem_calibrated, alpha_acc_zone)
    
    
#     plt.fill_between(x_axis, y_low, y_up, color='grey', alpha=0.4)
    plt.plot(x_axis, y_test, 'ro', markersize=4,  alpha=.6, label='True capacity')
#     plt.fill_between(x_axis, y_acc_zone_low, y_acc_zone_up, color='blue', alpha=0.4, label='Accuracy zone [-$\\alpha$, $\\alpha$]')
    
    plt.plot(x_axis, y_hat, 'o', color='k', markersize=4, alpha=.6, label='Predicted capacity')
    plt.fill_between(x_axis, y_low_calibrated, y_up_calibrated, color='orange', alpha=0.4, label='±2$\sigma$')

#     plt.errorbar(x_axis, y_hat, yerr=np.sqrt(V_IJ_unbiased), fmt='o')
    plt.legend(fontsize=20)
    plt.xlabel('Cycle', fontsize=20)
    if battery == 'oxford_dataset':
        plt.ylabel('Capacity [mAh]', fontsize=20)
    else:
        plt.ylabel('Capacity [Ah]', fontsize=20)

    plt.tick_params(axis='both', which='major', labelsize=20)


#     plt.grid(True)
    plt.show()
    
    # calculate accuracy 
    
    r2 = r2_score(y_test, y_hat)
    mape = np.round(np.mean(np.abs((y_test - y_hat) / y_test))*100,2)
    rmspe_calc = np.round(np.mean(np.sqrt(np.mean(np.square(((y_test - y_hat) / y_test)))))*100,2)
    mse = mean_squared_error(y_hat, y_test)
    
    
    print('Accuracy based on $R^2$: {}'.format(r2))
    print('Accuracy based on Mean Absolute Percentage Error: {}'.format(mape))
    print('Accuracy based on Root Mean Squared Percentage Error: {}'.format(rmspe_calc))
    print('Mean Square Error: {}'.format(mse))
    print('% of entriers falling in the accuracy zone: {}'.format(acc_zone_entries)) # note this is percentage
    print('Average probability mass of the prediciton PDF within the accuracy zone: {}'.format(beta))
    print('Ratio of early to late predictions when references to accurazy zone boundries: {}'.format(rlh))

    '''Calculate the calibrations score
       reference: Gaussian process regression for In-situ Capacity Estimation of Lithium-ion Batteries
       Rober R. Richardson, Birkl, Howey
       '''    
    CS = 1/len(X_test) * np.sum((np.abs(y_hat - y_test) < Z * sem))
    CS_calibrated = 1/len(X_test) * np.sum((abs(y_hat - y_test) < Z * sem_calibrated))
    SH = np.mean(sem_calibrated)
    
    print('Calibration score: {}'.format(CS))
    print('Calibration score calibrated model: {}'.format(CS_calibrated))
    print('Sharpness (mean std): {}'.format(SH))
    
    ''' Print the two capacities on the same plot without the cycle number'''
    fig2 = plt.figure(figsize=(10, 10))
    # acc zone display uniformly
    y = np.linspace(np.min(y_test), np.max(y_test), 100)
    y_up = y + alpha_acc_zone*y
    y_low = y - alpha_acc_zone*y
    ax1 = plt.axes()  # standard axes
    ax1.plot(y_test, y_test, 'ro', alpha=.7, zorder=10, label = 'True Capacity ($y^*$)', markersize=2)
    ax1.fill_between(y, y_low, y_up, color='green', alpha=0.6, zorder=10, label='Accuracy zone [-$\\alpha$, $\\alpha$]')
#     ax1.plot(y_test, y_hat, 'o', color='k', alpha=0.6, zorder=10, label='Predicted Capacity')
#     ax1.errorbar(y_test, y_hat, yerr=Z*sem, fmt='--o', color='blue', alpha=0.4, zorder=5, )
    markers, caps, bars = ax1.errorbar(y_test, y_hat, yerr=Z*sem_calibrated, ecolor='orange',capsize=2, capthick=2,
                                       fmt='--o', color='black',alpha=0.6, zorder=5, label='Predicted capacity ($\hat{y}^*$) ±2$\sigma$')

    # loop through bars and caps and set the alpha value
    [bar.set_alpha(0.4) for bar in bars]
    [cap.set_alpha(0.4) for cap in caps]
    ax1.invert_xaxis()

#     plt.plot(y_test, y_hat, 'o', color='orange', alpha=0.6)
#     plt.gca().invert_xaxis()
    plt.xlabel('True Capacity [Ah]', fontsize=20)
    if battery == 'oxford_dataset':
        plt.ylabel('Capacity [mAh]', fontsize=20)
    else:
        plt.ylabel('Capacity [Ah]', fontsize=20)
    plt.legend(fontsize=20)
    plt.tick_params(axis='both', which='major', labelsize=20)

#     plt.title('True vs. Predicted Capacity', fontsize=15)
#     plt.grid(True)
    ax2 = plt.axes([0.21, 0.21, 0.24, 0.24])
    percent_error = (y_hat-y_test)/y_test
    if battery in ['full_dataset', 'standford_dataset']:
        no_of_bins = 50 # 50-full_datset, 10 all other datasets
    else:
        no_of_bins = 10 # 50-full_datset, 10 all other datasets
    n, bins, patches = ax2.hist(percent_error, no_of_bins, edgecolor='white', color='black')
    
    for c, p in zip(bins, patches):
        if c >= alpha_acc_zone:
            plt.setp(p, 'facecolor', 'black')
        elif c <= -alpha_acc_zone:
            plt.setp(p, 'facecolor', 'black')
        else:
            plt.setp(p, 'facecolor', 'green') #mediumslateblue
    plt.axvline(0, color='red', linestyle='dashed')#, linewidth=1)
    plt.axvline(-alpha_acc_zone, color='green', linestyle='dashed')#, linewidth=1)
    plt.axvline(alpha_acc_zone, color='green', linestyle='dashed')#, linewidth=1)
    plt.axvline(0, color='red', linestyle='dashed')#, linewidth=1)
    plt.xlabel('% error: $(\hat{y}^*-y^*)/y^*$', fontsize=15)
    plt.ylabel('No. of entries', fontsize=15)
    plt.title('Histogram of % error wrt $\hat{y}^*$', fontsize=15)
    plt.tick_params(axis='both', which='major', labelsize=15)
#     plt.xlim([-1.2*np.max(abs(y_test-y_hat)), 1.2*np.max(abs(y_test-y_hat))])


    plt.figure(figsize=(10,10))
    plt.plot(abs(y_test-y_hat), sem_calibrated, '*')
    plt.ylabel('Variance')
    plt.xlabel('Error')

#     ax2.text(0.1, .5, r'$-\alpha$')
    
    return fig1, fig2, r2, mape, rmspe_calc, mse, acc_zone_entries, beta, rlh, CS_calibrated, SH

# Load data

In [ ]:


# test data
# data_test = pd.read_csv(path + 'data_test_fsed__adversarial' + battery +'.csv', index_col=0)
data_test = pd.read_csv()
X_test_complete = data_test.drop(['Discharge_Q', 'Group'], axis=1)
y_test_complete = data_test.Discharge_Q
data_test_4_group = data_test.Group

# load relevant battery dataset for training the algorithm
if adversarial:
    data_train = pd.read_csv()
else:
    data_train = pd.read_csv()
    
# train data used to train the model
# data_train = pd.read_csv(path + 'data_train_fsed_'+ battery + '.csv', index_col=0)
X_train = data_train.drop(['Discharge_Q', 'Group'], axis=1)
y_train = data_train.Discharge_Q

# re-calibration data


data_calibration = pd.read_csv()



In [ ]:
print('Number of features selected based on RFE-CV: {} for battery dataset {}'.format(len(list(X_train)), battery))



# Expected accuracy


In [ ]:
'''
Use +/- alpha accuracy zone as per reference: "On Applying the Prognostics Performance Metrics by Saxena NASA"
'''
alpha_acc_zone = 0.015 # how to determine the alpha value - 1.5% is the arbina error, but how to justify the rest?
Z = st.norm.ppf(1-(1-0.9)/2) # calculate z-value for 90% confidence
STD = alpha_acc_zone/Z
var_expected = STD**2
print('Expected variance based on domain specific selection: {}'.format(var_expected))
print('Z-value used: {}'.format(Z))

In [ ]:
'''load models'''
# random forest model
RF_model_filenname = path_models + 'RF_model_adversarial_'+ battery +'.sav'
# RF_model_filenname = path_models + 'RF_model_'+ battery +'.sav'
RF = pickle.load(open(RF_model_filenname, 'rb'))
RF_model = RF.best_estimator_

# bayesian ridge 
Bayes_Ridge_filename = path_models + 'Bayesian_Ridge_adversarial_'+ battery +'.sav'
# Bayes_Ridge_filename = path_models + 'Bayesian_Ridge_'+ battery +'.sav'
Bayes_Ridge = pickle.load(open(Bayes_Ridge_filename, 'rb'))
Bayes_Ridge_model = Bayes_Ridge.best_estimator_

# Gaussian process model
GPR_model_filenname = path_models + 'GPR_model_'+ battery +'.sav'
GPR = pickle.load(open(GPR_model_filenname, 'rb'))
GPR_model = GPR.best_estimator_



In [ ]:
# predict with a selected model - HARD CODED

model_name = 'GPR_model' # RF_model, Bayes_Ridge_model, GPR_model
model = GPR_model # RF_model, Bayes_Ridge_model, GPR_model





In [ ]:
data_calibration.head()

In [ ]:


r2_vec = []
mape_vec = []
rmspe_vec = []
mse_vec = []
acc_zone_percentage_vec = []
beta_vec = []
rlh_vec = []
avg_calibration_vec = []
SH_vec = []

for i in data_test_4_group.unique():
    
    group_calibration = [] # specify the group of cells on which to calibrate the model 

    # load calibration data per group
    data_calibration_group = data_calibration[data_calibration.Group.isin(group_calibration)]

    X_calibration = data_calibration_group.drop(['Discharge_Q', 'Group'], axis=1)
    y_calibration = data_calibration_group.Discharge_Q

    calibration_model = calibration_isotonic_regression_model(model_name, model, X_calibration, y_calibration, X_train)
    print('Calibration done!')


    data_group = data_test_4_group[data_test_4_group==i]
    min_group = data_group.index[0]
    max_group = data_group.index[-1]
    
    X_test = X_test_complete[min_group:max_group]
    X_test.reset_index(inplace=True, drop=True)
    y_test = y_test_complete[min_group:max_group]
    y_test.reset_index(inplace=True, drop=True)
    
    if model_name in ['Bayes_Ridge_model', 'GPR_model']:
        y_hat,  sem = model.predict(X_test, return_std=True)
        
    elif model_name == 'RF_model':
        y_hat = model.predict(X_test)
        sem = np.sqrt(fci.random_forest_error(RF_model, X_train, X_test))

    else:
        y_hat,  sem = model.predict(X_test)

    prob_per_int_y_test, prob_y_test, prob_y_test_expected, prob = count_entries_per_interval(y_test, y_hat, sem)
    prob_model_y_test = predict_prob(y_test, y_hat, sem)
    
    prob_test_calibrated = calibrated_prob(calibration_model, prob_model_y_test)

    sem_calibrated = std_calibrated(y_test, y_hat, prob_test_calibrated)
    prob_per_int_y_test_calibrated, prob_y_test_calibrated, prob_y_test_expected, prob = count_entries_per_interval(y_test, y_hat, sem_calibrated)

    '''Plotting calibration only'''
    
    # prob_calibration_calibrated = ir.transform(prob_model_y_calibration)
    plt.figure()
#     plt.title('Test data')
    plt.plot(prob, prob, 'k--')
    plt.plot(prob, prob_per_int_y_test, 'b*--')
    plt.plot(prob, prob_per_int_y_test_calibrated, '*--', c='orange')
    plt.xlabel('Expected Cofidence Level', fontsize=20)
    plt.ylabel('Observed Confidence Level', fontsize=20)
    plt.legend(['Ideal calibration', 'Uncalibrated', 'Calibrated'], fontsize=20)
    plt.grid(True)

    
    print('------------------------Battery group: {}'.format(i))
    # function to plot the results
    fig1, fig2, r2, mape, rmspe, mse, acc_zone_percentage, beta, rlh, CS, SH = plot_results(y_hat, y_test, sem, sem_calibrated, Z)
       
   # plot reliability curve for the paper
    if r2 < 0 or len(prob)<11: # plot reliability curves for an r2 better greate than 0
        pass
    else:
        # note: if there is not suffiecint data in index prob rounding should go to 1
        index = []
        round_value = 2   # <---- for more granularity also changethe round value
        for q in np.round(np.linspace(0, 1, 11), round_value):
            
#                                                                      | CHANGE THIS LOCATION TO OBTAIN A MORE ACCURATE CALIBRATION CURVE FOR  THE PLOTS
            index.append(np.where(np.round(prob, round_value) == q)[0][2]) # was [0][1] i.e. checking second element, used to raound ro 2
                                                                    #[0][2]
        from operator import itemgetter 
        prob_expected = itemgetter(*index)(prob)
        prob_observed_uncalibrated = itemgetter(*index)(prob_per_int_y_test)
        prob_observed_calibrated = itemgetter(*index)(prob_per_int_y_test_calibrated)
        fig3 = plt.figure(figsize=(10,10))
    #     plt.title('Test data')
        plt.plot(prob_expected, prob_expected, 'k--', markersize=2)
        plt.plot(prob_expected, prob_observed_uncalibrated, 'bo--', markersize=10, linewidth=6)
        plt.plot(prob_expected, prob_observed_calibrated, 'o--', c='orange', markersize=10, linewidth=6)
        plt.xlabel('Expected Cofidence Level', fontsize=20)
        plt.ylabel('Observed Confidence Level', fontsize=20)
        plt.legend(['Ideal calibration', 'Uncalibrated', 'Calibrated'], fontsize=20)
        plt.tick_params(axis='both', which='major', labelsize=20)
    #     plt.grid(True) 

#     '''Save figures'''    
    print('saved askjahsdkjhsadkashdhasjh')
    fig3.savefig(path_to_save_figures + "cell_no_" + str(int(i))+ '_' + model_name +  ".pdf", bbox_inches='tight')
    fig1.savefig(path_to_save_figures + "cell_no_"+ str(int(i))+ '_' + model_name + ".pdf", bbox_inches='tight')
    fig2.savefig(path_to_save_figures + "cell_no_" + str(int(i))+ '_' + model_name + ".pdf", bbox_inches='tight')

    
    # average results
    cycles_w_neg_r2 = []
    if r2 > 0:
        cycles_w_neg_r2.append(i)
        
    r2_vec.append(r2)
    mape_vec.append(mape)
    rmspe_vec.append(rmspe)
    mse_vec.append(mse)
    acc_zone_percentage_vec.append(acc_zone_percentage)
    beta_vec.append(beta)
    rlh_vec.append(rlh)
    avg_calibration_vec.append(CS)
    SH_vec.append(SH)

# avergae values across dataset
r2_mean = np.mean(r2_vec)
mape_mean = np.mean(mape_vec)
rmspe_mean = np.mean(rmspe_vec)
mse_mean = np.mean(mse_vec)
acc_zone_percentage_mean = np.mean(acc_zone_percentage_vec)
beta_mean = np.mean(beta_vec)
rlh_mean = np.mean(rlh_vec)
calibration_mean = np.mean(avg_calibration_vec)
SH_mean = np.mean(SH_vec)

print('#############################################################################################')
print('Mean accuracy based on $R^2$: {}'.format(r2_mean))
print('Mean accuracy based on Mean Absolute Percentage Error: {}'.format(mape_mean))
print('Mean accuracy based on Root Mean Squared Percentage Error: {}'.format(rmspe_mean))
print('Mean Mean Square Error: {}'.format(mse_mean))
print('Mean % of entriers falling in the accuracy zone: {}'.format(acc_zone_percentage_mean)) # note this is percentage
print('Mean average probability mass of the prediciton PDF within the accuracy zone: {}'.format(beta_mean))
print('Mean percentage of early predictions when references to accurazy zone boundries: {}'.format(rlh_mean))
print('Mean calibration score calibrated model: {}'.format(calibration_mean))
print('Mean sharpness: {}'.format(SH_mean))
print('#############################################################################################')

 

In [ ]:
plt.fill_between(y_test, y_test+alpha_acc_zone*y_test , (y_test-alpha_acc_zone*y_test))

